In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from tensorflow.keras.utils import to_categorical

In [ ]:
X = train.drop(['credit'],axis=1)
y = train[['credit']]

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

random.seed(42)
xgb_models={}
xgb_outcomes = []

for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    xgb=XGBClassifier(max_depth=18,
                      n_estimators=1000,
                      min_child_weight=0.5, 
                      min_child_samples=3, 
                      learning_rate =0.01,
                      num_leaves=300,
                      colsample_bytree=0.3)
    evals = [(X_valid,y_valid)]
    xgb.fit(X_train, y_train,
               early_stopping_rounds=30, 
               eval_metric='mlogloss',
               eval_set=evals,
               verbose=100)
    
    predictions = xgb.predict_proba(X_valid)
    logloss = log_loss(to_categorical(y_valid['credit']), predictions)
    xgb_outcomes.append(logloss)

    xgb_models[fold]=xgb
    
    print(f'================================================================================\n\n')

In [ ]:
mean_outcome = np.mean(xgb_outcomes)
print(f"Mean:{mean_outcome}")

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += xgb_models[fold].predict_proba(test)/5

In [ ]:
submit.to_csv('submit5.csv', index=False) 